# **Setup**

* https://towardsdatascience.com/modern-route-optimization-with-python-fea87d34288b
* https://www.kaggle.com/datasets/starbucks/store-locations

In [1]:
from IPython.display import clear_output

# !python -m pip install folium
clear_output()
# !python -m pip install osmnx
clear_output()

In [30]:
import os 
from warnings import filterwarnings
filterwarnings('ignore')

import pandas as pd 
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# import plotly.express as px

# For Geospatial Visualization
import folium 
from folium import plugins

# For Routing 
import osmnx as ox
import ortools as ot

In [3]:
Folder_Input = 'Input Data'

if not os.path.exists(Folder_Input):
    os.mkdir(Folder_Input)

# **Data Import**

## **Data Download**

In [4]:
os.chdir(Folder_Input)

!kaggle datasets download -d starbucks/store-locations
os.chdir('..')


  0%|          | 0.00/1.10M [00:00<?, ?B/s]
 91%|█████████ | 1.00M/1.10M [00:00<00:00, 2.04MB/s]
100%|██████████| 1.10M/1.10M [00:00<00:00, 2.22MB/s]


## **Extract Zip File**

In [5]:
def Extract_Zip(Zip_Filepath, Destination_Path, Delete_Zip):
    from zipfile import ZipFile
    with ZipFile(Zip_Filepath, 'r') as zip:
        zip.extractall(Destination_Path)
        if Delete_Zip:
            os.remove(Zip_Filepath)

In [6]:
Extract_Zip(Zip_Filepath=f'{Folder_Input}/store-locations.zip',
            Destination_Path=f'{Folder_Input}',Delete_Zip=False)

In [7]:
os.remove(f'{Folder_Input}/store-locations.zip')

In [8]:
df = pd.read_csv(f'{Folder_Input}/directory.csv')

print(df.shape)
print(df.columns)
df.head()

(25600, 13)
Index(['Brand', 'Store Number', 'Store Name', 'Ownership Type',
       'Street Address', 'City', 'State/Province', 'Country', 'Postcode',
       'Phone Number', 'Timezone', 'Longitude', 'Latitude'],
      dtype='object')


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


## **Data Review**

In [9]:
df.isna().sum()

Brand                0
Store Number         0
Store Name           0
Ownership Type       0
Street Address       2
City                15
State/Province       0
Country              0
Postcode          1522
Phone Number      6861
Timezone             0
Longitude            1
Latitude             1
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25600 entries, 0 to 25599
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           25600 non-null  object 
 1   Store Number    25600 non-null  object 
 2   Store Name      25600 non-null  object 
 3   Ownership Type  25600 non-null  object 
 4   Street Address  25598 non-null  object 
 5   City            25585 non-null  object 
 6   State/Province  25600 non-null  object 
 7   Country         25600 non-null  object 
 8   Postcode        24078 non-null  object 
 9   Phone Number    18739 non-null  object 
 10  Timezone        25600 non-null  object 
 11  Longitude       25599 non-null  float64
 12  Latitude        25599 non-null  float64
dtypes: float64(2), object(11)
memory usage: 2.5+ MB


In [11]:
df.describe()

,Longitude,Latitude
count,25599.000000,25599.000000
mean,-27.872234,34.793016
std,96.844046,13.342332
min,-159.460000,-46.410000
25%,-104.665000,31.240000
50%,-79.350000,36.750000
75%,100.630000,41.570000
max,176.920000,64.850000


# **Data Cleaning**

In [12]:
df.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


In [13]:
# Remove Postcode & Phone Number Column
df.drop(['Postcode', 'Phone Number','Timezone'], axis=1, inplace=True)

In [14]:
print(df.shape)
df.head()

(25600, 10)


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,54.54,24.51


# **Data Exploration**

## **Brand Count**

In [15]:
df['Brand'].value_counts()

Brand
Starbucks                25249
Teavana                    348
Evolution Fresh              2
Coffee House Holdings        1
Name: count, dtype: int64

## **Ownership Count**

In [16]:
df['Ownership Type'].value_counts()

Ownership Type
Company Owned    11932
Licensed          9375
Joint Venture     3976
Franchise          317
Name: count, dtype: int64

## **Country Count**

In [17]:
# Identify The Country With The Highest Number Of Stores
df['Country'].value_counts().head(5)

Country
US    13608
CN     2734
CA     1468
JP     1237
KR      993
Name: count, dtype: int64

## **US City Count**

In [18]:
# Identify US City With Highest Number Of Stores 
df[df['Country']=='US']['City'].value_counts().head(5)

City
New York     232
Chicago      180
Seattle      156
Las Vegas    156
Houston      154
Name: count, dtype: int64

## **City With Most Stores**

In [19]:
df.columns

Index(['Brand', 'Store Number', 'Store Name', 'Ownership Type',
       'Street Address', 'City', 'State/Province', 'Country', 'Longitude',
       'Latitude'],
      dtype='object')

In [20]:
df.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,54.54,24.51


In [21]:
df.groupby(by='City')['Store Number'].agg('count').sort_values(ascending=False).head(5)

City
上海市         542
Seoul       243
北京市         234
New York    232
London      216
Name: Store Number, dtype: int64

In [22]:
df[df['City']=='上海市']

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Longitude,Latitude
2518,Starbucks,24754-238132,汇暻生活广场店,Joint Venture,"黄浦区, 汇暻生活广场, 四川北路1689号110、202室",上海市,31,CN,121.48,31.20
2519,Starbucks,28083-249527,嘉杰广场店,Joint Venture,"虹口区, 牡丹江路1569号F1层F127铺位",上海市,31,CN,121.48,31.26
2520,Starbucks,25485-226668,宝乐汇店,Joint Venture,"上海市, 长寿路828，846号",上海市,31,CN,121.49,31.40
2521,Starbucks,27243-246386,中港汇店,Joint Venture,"普陀区, 中港汇大厦, 荣乐西路790号1F-14",上海市,31,CN,121.43,31.24
2522,Starbucks,24722-238648,松江新理想广场店,Joint Venture,"松江区, 新理想广场, 国际机场交通中心地铁2号线西侧廊道靠中步行道段A标段01号商铺",上海市,31,CN,121.20,31.02
...,...,...,...,...,...,...,...,...,...,...
3055,Starbucks,23329-229146,汇商大厦店,Joint Venture,"浦东新区, 金丰路447号",上海市,31,CN,121.55,31.22
3056,Starbucks,31776-114987,金丰路店,Joint Venture,"闵行区, 金丰路, 花木路1378号",上海市,31,CN,121.28,31.20
3057,Starbucks,15191-153628,浦东嘉里城店,Joint Venture,"浦东新区, 浦东嘉里城商场一层L110单元, 新金桥路1088号B100室",上海市,31,CN,121.56,31.21
3058,Starbucks,32235-124151,中惠广场店,Joint Venture,"浦东新区, 中惠广场, 福州路465号2楼",上海市,31,CN,121.61,31.25


# **City Data**

## **Filter City Data**

In [23]:
# Select City Name
City_Name = 'New York'

In [24]:
City = df[df['City']==City_Name]

print(City.shape)
print(City.columns)
City.head()

(232, 10)
Index(['Brand', 'Store Number', 'Store Name', 'Ownership Type',
       'Street Address', 'City', 'State/Province', 'Country', 'Longitude',
       'Latitude'],
      dtype='object')


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Longitude,Latitude
21129,Starbucks,818-227125,80th & Second,Company Owned,245 East 80th St,New York,NY,US,-73.95,40.77
21130,Starbucks,14318-119964,Nassau & Fulton,Company Owned,130 Fulton Street,New York,NY,US,-74.01,40.71
21131,Starbucks,7670-36979,49th & 8th,Company Owned,"825 Eighth Avenue, W-9",New York,NY,US,-73.99,40.76
21132,Starbucks,7626-33939,Spring and Varick,Company Owned,"150 Varick Street, The Lyric Building",New York,NY,US,-74.01,40.73
21133,Starbucks,7699-45959,95th & Broadway,Company Owned,2521 Broadway,New York,NY,US,-73.97,40.79


In [25]:
City.drop(columns=['Brand','Ownership Type','Street Address','State/Province'],inplace=True)
City.rename(columns={'Store Number':'StoreID'},inplace=True)
City.reset_index(inplace=True,drop=True)

print(City.shape)
print(City.columns)
City.head()

(232, 6)
Index(['StoreID', 'Store Name', 'City', 'Country', 'Longitude', 'Latitude'], dtype='object')


,StoreID,Store Name,City,Country,Longitude,Latitude
0,818-227125,80th & Second,New York,US,-73.95,40.77
1,14318-119964,Nassau & Fulton,New York,US,-74.01,40.71
2,7670-36979,49th & 8th,New York,US,-73.99,40.76
3,7626-33939,Spring and Varick,New York,US,-74.01,40.73
4,7699-45959,95th & Broadway,New York,US,-73.97,40.79


## **Visualize Store Locations**

In [35]:
City['Longitude'].mean()

-73.98275862068965

In [36]:
City['Latitude'].mean()

40.75327586206896

In [48]:
Map01 = folium.Map(location=(City['Latitude'].mean(),City['Longitude'].mean()),zoom_start=12)

for i in range(City.shape[0]):
    folium.CircleMarker(location=(City['Latitude'][i],City['Longitude'][i]),radius=5,color='blue',fill=True).add_to(Map01)

Map01